## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [15]:
import pandas as pd
data = pd.read_csv('/datasets/credit_scoring_eng.csv')
creditScore = pd.DataFrame(data=data)

### Conclusion

### Step 2. Data preprocessing

### Processing missing values

In [17]:
# handle negative or missing children
creditScore['children'] = creditScore['children'].replace(-1,0)
creditScore['children'] = creditScore['children'].replace(20,0)
creditScore['children'].fillna(0,inplace=True)

# handle missing days employee
creditScore['days_employed'].fillna(0,inplace=True)

# handle missing total income
creditScore['total_income'].fillna(0,inplace=True)
print(creditScore.duplicated().sum())

54


In the processing missing values step, I identified three columns that had missing values: 'children, 'days_employed' and 'total_income'. In the 'children' column, there were rows that contained a negative value and rows that contained an outlier value. Assuming both of these are data entry errors, as it is impossible to have a negative amount of children and it is very unlikely to have 20 children, I replaced the values in these rows with 0. Any other rows with missing values was giving a default value of 0 children. In the 'days_employed' and 'total_income' columns, I replaced null values with 0 in order to fill in those missing values. 

### Conclusion

### Data type replacement

In [4]:
# change float type to int for days_employed
creditScore['days_employed'] = creditScore['days_employed'].astype(int)

for days in creditScore['days_employed']:
    if days < 0:
        creditScore['days_employed'] = creditScore['days_employed'].replace(days, 0)
        
creditScore['years_employed'] = (creditScore['days_employed']/365).astype(int).round()

# change float type to int for total_income
creditScore['total_income'] = creditScore['total_income'].astype(int)

#handle duplicates in education
creditScore['education'] = creditScore['education'].str.lower()

#handle duplicates in purpose
import nltk
from nltk.stem import WordNetLemmatizer

wordnet_lemmma = WordNetLemmatizer()

for purpose in creditScore['purpose']:
    words = nltk.word_tokenize(purpose)

    if 'education' in words or 'university' in words or 'educated' in words:
        creditScore['purpose'].replace(purpose, 'education',inplace=True)
    
    if 'car' in words or 'cars' in words:
        creditScore['purpose'].replace(purpose,'car',inplace=True)
        
    if 'house' in words or 'housing' in words or 'estate' in words or 'property' in words:
        creditScore['purpose'].replace(purpose,'real estate',inplace=True)

    if 'wedding' in words:
        creditScore['purpose'].replace(purpose,'wedding',inplace=True)

#print(creditScore['purpose'].unique())


['real estate' 'car' 'education' 'wedding']
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
dtype: int64


In this data type replacement step, I started my handling the 'days_employed' column. I believe that this column is quite corrupted as there are many empty values or absurdly high values. To handle this column, I changed the type of the column from float to int, then I changed any negative value for this column to 0 (if you have been employed for a negative amount of time, then you have not been employed for that amount of time and therefore, your days employed are 0) and then I went through each value to change the number from days to years for ease of readibility then stored this new years value in a new column, years_employed. Even after this cleanup, there are still very strange values - looking at the first 30 rows in this column yields either 0 years employed or between 932 to 1096 years of employment, which is not possible. 

For the 'total_income' column, I changed the type of the column from float to int to make it easier to read the numbers. For the 'education' column, there were many of the same values but with different casing (for example, 'secondary education' and 'Secondary Education' are the same value but because of the capitilization difference, are stored differently). I used the str.lower() method to lower case all of the values which removes this unnecessary repetition.

The 'purpose' column was very problematic because there were many purposes that were essentially the same but because of the different wording, was stored differently. For example, 'to have a wedding' and 'wedding ceremony' are the same purpose but worded differently so will be handed as different reasons. We don't want this so I used lemmatization to find all unique identifiable words in the raw purpose output and from that, categorized the purpose accordingly. 

### Conclusion

### Processing duplicates

In [5]:
#print(creditScore.duplicated().sum())
creditScore = creditScore.drop_duplicates().reset_index(drop=True)

In this processing duplicates step, I identified 408 duplicates in the data. I decided to drop all of these duplicates using the drop_duplicated() method.

### Conclusion

### Categorizing Data

### Conclusion

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [7]:
kidsData = creditScore.groupby('children')
print(kidsData['debt'].describe())
print(kidsData['debt'].count()/kidsData['debt'].sum())

            count      mean       std  min  25%  50%  75%  max
children                                                      
0         13949.0  0.076708  0.266137  0.0  0.0  0.0  0.0  1.0
1          4751.0  0.093454  0.291098  0.0  0.0  0.0  0.0  1.0
2          2039.0  0.095145  0.293487  0.0  0.0  0.0  0.0  1.0
3           329.0  0.082067  0.274885  0.0  0.0  0.0  0.0  1.0
4            40.0  0.100000  0.303822  0.0  0.0  0.0  0.0  1.0
5             9.0  0.000000  0.000000  0.0  0.0  0.0  0.0  0.0
children
0    13.036449
1    10.700450
2    10.510309
3    12.185185
4    10.000000
5          inf
Name: debt, dtype: float64


I started by grouping the data by the 'children' column. 

There are 13949 rows with 0 children, 4751 rows with 1 child, 2039 rows with 2 children, 329 rows with 3 children, 40 rows with 4 children and 9 rows with 5 children. 

Looking at the 'debt' column within this data grouping, there are 1070 rows with debt and 0 children, 444 rows with debt and 1 child, 194 rows with debt and 2 children, 27 rows with debt and 3 children, 4 rows with debt and 4 children and 0 rows with debt and 5 children.

Putting these two numbers together: 13.0% of rows with 0 children have debt, 10.7% of rows with 1 child have debt, 10.5% of rows with 2 children have debt, 12.1% of rows with 3 children have debt, 10% of with 4 children have debt and 0% of rows with 5 children have debt.

From this data, we can conclude that there is a relation between having kids and paying debt on time. The rows with more children are more likely to repay their loans on time, with the exception of the rows with 3 children. 

In [8]:
# reviewer's code 

creditScore.pivot_table(index = 'children', values = 'debt', aggfunc = ['count', 'mean'])

,count,mean
,debt,debt
children,,
0,13949,0.076708
1,4751,0.093454
2,2039,0.095145
3,329,0.082067
4,40,0.100000
5,9,0.000000


### Conclusion

- Is there a relation between marital status and repaying a loan on time?

In [9]:
maritalData = creditScore.groupby('family_status')
print(maritalData['debt'].describe())
print(maritalData['debt'].count()/maritalData['debt'].sum())

                     count      mean       std  min  25%  50%  75%  max
family_status                                                          
civil partnership   4124.0  0.094083  0.291980  0.0  0.0  0.0  0.0  1.0
divorced            1193.0  0.071249  0.257348  0.0  0.0  0.0  0.0  1.0
married            12074.0  0.076942  0.266511  0.0  0.0  0.0  0.0  1.0
unmarried           2784.0  0.098420  0.297935  0.0  0.0  0.0  0.0  1.0
widow / widower      942.0  0.066879  0.249945  0.0  0.0  0.0  0.0  1.0
family_status
civil partnership    10.628866
divorced             14.035294
married              12.996771
unmarried            10.160584
widow / widower      14.952381
Name: debt, dtype: float64


I started by grouping the data by the 'family_status' column.

Looking at the data: 388/4124 = 14.9% of those who are widows have debt, 14.0% of those who are divorced have debt, 13% of those who are married have debt,10.6% of those in a civil partnership have debt, 10.1% of those who are unmarried have debt.

From this data, we can conclude that there is a relation between martial status and repaying the debt on time. Those who either have been married (widows, divorced) or are married are more likely to have debt than those who are unmarried (either single or in a civil partnership). 

### Conclusion

- Is there a relation between income level and repaying a loan on time?

In [10]:
incomeData = creditScore.groupby('income_type')
print(incomeData['debt'].describe())
print(incomeData['debt'].count()/incomeData['debt'].sum())

                               count      mean       std  min   25%  50%  \
income_type                                                                
business                      5032.0  0.074722  0.262968  0.0  0.00  0.0   
civil servant                 1450.0  0.059310  0.236286  0.0  0.00  0.0   
employee                     10899.0  0.097165  0.296196  0.0  0.00  0.0   
entrepreneur                     2.0  0.000000  0.000000  0.0  0.00  0.0   
paternity / maternity leave      1.0  1.000000       NaN  1.0  1.00  1.0   
retiree                       3730.0  0.057909  0.233602  0.0  0.00  0.0   
student                          1.0  0.000000       NaN  0.0  0.00  0.0   
unemployed                       2.0  0.500000  0.707107  0.0  0.25  0.5   

                              75%  max  
income_type                             
business                     0.00  1.0  
civil servant                0.00  1.0  
employee                     0.00  1.0  
entrepreneur                 0.00 

I started by grouping the data by the 'income_type' column.

Looking at the data: 376/5032 = 13.3% of those who own a business have debt, 86/1450 = 16.8% of those who are civil servants have debt, 1059/10899 = 10.3% of those who are employees have debt, neither of the two entrepreneurs in the data have debt, the 1 row on paternity/maternity leave have debt, 216/3730 = 17.2% of retirees have debt, the 1 student in the data does not have debt, and only 1 of the 2 unemployed in the data have debt. 

From this data, we can conclude that there is a relation income level and repaying the loan on time. Those who either were employed (retired) or are employed by an external entity (business, civil servent, employee) have high percentage of debt than those who are either self employed (entrepreneur), on paternity / maternity leave, a student or unemployed. 

### Conclusion

- How do different loan purposes affect on-time repayment of the loan?

In [11]:
purposeData = creditScore.groupby('purpose')
print(purposeData['debt'].describe())
print(purposeData['debt'].count() / purposeData['debt'].sum())

               count      mean       std  min  25%  50%  75%  max
purpose                                                          
car           4271.0  0.094123  0.292034  0.0  0.0  0.0  0.0  1.0
education     3964.0  0.093340  0.290945  0.0  0.0  0.0  0.0  1.0
real estate  10576.0  0.073846  0.261533  0.0  0.0  0.0  0.0  1.0
wedding       2306.0  0.080659  0.272370  0.0  0.0  0.0  0.0  1.0
purpose
car            10.624378
education      10.713514
real estate    13.541613
wedding        12.397849
Name: debt, dtype: float64


I started by grouping the data by the 'purpose' column.

Looking at the data: 781/10576 = 13.5% of those with debt took it out for real estate purpoes, 186/2306 = 12.4% of those with debt took it out for wedding purposes, 370/3964 = 10.7% of those with debt took it out for education purposes and 402/4271 = 10.6% of those with debt took it out for car purchase purposes. 

From this data, we can conclude that most people take out loans for real estate purposes (50% of people who took out debt took it out for real estate purposes). As a result, the number of those who have debt and took out that debt for real estate purposes is going to be the highest. Those who took out loans for real estate or wedding purposes are less likely to repay their loans on time than those who took out loans for car or education purposes.

### Conclusion

### Step 4. General conclusion



From this data, we can conclude that there is a relation income level and repaying the loan on time. Those who either were employed (retired) or are employed by an external entity (business, civil servent, employee) have high percentage of debt than those who are either self employed (entrepreneur), on paternity / maternity leave, a student or unemployed. 


In conclusion:
- Those who have children are generally more likely to repay their loans on time, compared to those who do not have children.
- Those who have been married or are currently maried are generally more likely to have debt than those who have never been married or are not married.
- Most people take out loans for real estate purposes. Those who take out loans for real estate or wedding purposes are less likely to repay their loans on time.
- Those who either were employed (retired) or are currently employed by an external entity (business, civil servant,employee) are more likely to have debt than those who are either self-employed (entrepreneur), on paternity / maternity leave, a student or unemployed.